**Loading necessary libraries**

In [1]:
# Imports
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal
import os

print("✅ All imports successful")

✅ All imports successful


In [2]:
# Load API key
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found! Please set it in your .env file.")

print("✅ API key loaded")

✅ API key loaded


In [3]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.5,
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")

✅ LLM initialized: gpt-4o-mini


**Loading and Processing Document**

In [20]:
document_path = r"agent\data"
document_list = os.listdir(document_path)

# Initialize a master list to hold pages from ALL PDFs
all_pages = []
for i, file in enumerate(document_list, start=1):
    file_path = os.path.join(document_path, file)
    
    # Check if file exists
    if not os.path.exists(file_path):
        print(f"⚠️ File not found: {file_path}")
        print("Please update the file_path variable with your PDF file.")
    else:
        # Load the PDF
        loader = PyPDFLoader(file_path)
        current_file_pages = []
        
        # Load pages (async loading)
        async for page in loader.alazy_load():
            current_file_pages.append(page)
            
         # Add these pages to the master list
        all_pages.extend(current_file_pages)   
        
        print(f"{i} ✅ Loaded {len(current_file_pages)} pages from {file}\n")
        
print(f"\nTotal pages loaded across all files: {len(all_pages)}\n")

1 ✅ Loaded 4 pages from Amendment_of_the_National_Law_on_NELFUND.pdf

2 ✅ Loaded 11 pages from Explanatory_Memorandum_For_Students-Loans_Act_2023.pdf

3 ✅ Loaded 4 pages from FAQs_On_NELFUND_Student_Loan.pdf

4 ✅ Loaded 14 pages from Guidelines_For_Nelfund_Loan.pdf

5 ✅ Loaded 4 pages from Terms&Conditions_of_Students_Loan.pdf


Total pages loaded across all files: 37



**Split into Chunks**

In [21]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # Characters per chunk
    chunk_overlap=100     # Overlap to preserve context
)

# Split documents
doc_splits = text_splitter.split_documents(all_pages)

print(f"✅ Created {len(doc_splits)} chunks")
print(f"\nSample chunk:")
print(f"{doc_splits[0].page_content[:200]}...")

✅ Created 55 chunks

Sample chunk:
Amendment  of  the  National  Law  on  NELFUND  
 
 
President
 
Bola
 
Tinubu,
 
on
 
Wednesday,
 
April
 
3,
 
2024,
 
signed
 
into
 
law
 
the
 
Student
 
Loans
 
(Access
 
to
 
Higher
 
Education...


**Create Vector Store (Chroma)**

In [22]:
# Initialize embeddings (using OpenAI)
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=openai_api_key
)

print("✅ Embeddings model initialized")

✅ Embeddings model initialized


In [23]:
# Create Chroma vector store
chroma_path = "agent\database\chroma_db"


# Create vector store from documents
vectorstore = Chroma(
    collection_name="agentic_rag_docs",
    persist_directory=chroma_path,
    embedding_function=embeddings
)

# Add documents
vectorstore.add_documents(documents=doc_splits)

print(f"✅ Vector store created with {len(doc_splits)} chunks")
print(f"   Persisted to: {chroma_path}")

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Welcome Sir\AppData\Local\Temp\ipykernel_15136\1187449133.py:2: SyntaxWarning: invalid escape sequence '\d'
  chroma_path = "agent\database\chroma_db"


✅ Vector store created with 55 chunks
   Persisted to: agent\database\chroma_db


**Test Retrieval**

In [25]:

# Test the vector store
test_query = "What is Nelfund?"
test_results = vectorstore.similarity_search(test_query, k=2)

print(f"Query: {test_query}")
print(f"\nTop result:")
print(f"{test_results[0].page_content[:200]}...")
print(f"\n✅ Retrieval working!")

Query: What is Nelfund?

Top result:
2  
education sector through NELFUND, Nigeria is unlocking immense potentials 
by paving ways for brighter future generations. 
1.3 Objectives of NELFUND 
1.3.1 Enhance Access to Higher Education 
To ...

✅ Retrieval working!


**Create Retrieval Tool**

In [26]:
@tool
def retrieve_nelfund_docs(query: str) -> str:
    """
    Search for relevant documents in the knowledge base.
    Search for official NELFUND policy documents, eligibility, and application guides.
    Use ONLY for factual questions about student loans.
    
    Use this tool when you need information from the document collection
    to answer the user's question. Do NOT use this for:
    - General knowledge questions
    - Greetings or small talk
    - Simple calculations
    
    Args:
        query: The search query describing what information is needed
        
    Returns:
        Relevant document excerpts that can help answer the question
    """
    # Using MMR (Max Marginal Relevance) for diversity
    retriever = vectorstore.as_retriever(
        search_type="mmr", 
        search_kwargs={"k": 3})
    
    # Retrieve documents
    results = retriever.invoke(query)
    
    if not results:
        return "No relevant NELFUND documents found."
    
    # Format results
    formatted = "\n\n".join(
        f"Content: {doc.page_content}\nSource: {doc.metadata['source']}"
        for doc in results
    )
    return formatted

In [29]:
# Test tool directly
test_result = retrieve_nelfund_docs.invoke({"query": "When does repayment starts?"})
print(f"Tool result (first 300 chars):\n{test_result[:300]}...")

Tool result (first 300 chars):
Content: 10  
LOAN REPAYMENT 
charges payment, such as falsifying student information/records or 
colluding with students to obtain loans for ineligible purposes. 
▪ The institution fails to refund disbursed institutional charges in instances 
stipulated in Section 5.5 above. 
▪ The institution fail...


### **Building Agentic RAG System**

**System Prompt**

In [30]:
system_prompt = SystemMessage(content="""You are a helpful assistant with access to a document retrieval tool.
    You are the NELFUND Navigator, an intelligent assistant for Nigerian students.
RETRIEVAL DECISION RULES:

DO NOT retrieve for:
- Greetings: "If the user greets you (e.g., "Hello", "Hi", "How are you"), reply warmly without using tools.)
- Questions about your capabilities: "What can you help with?", "What do you do?"
- Simple math or general knowledge: "What is 2+2?"
- Casual conversation: "Thank you", "Goodbye"


DO retrieve for:
- Questions asking for specific information that would be in documents
- If the user asks about loans, eligibility, or NELFUND, use the 'retrieve_nelfund_docs' tool.
- Any question where citing sources would improve the answer

CITATION RULES:
- You MUST cite your sources using the format.
- For example: "Repayment starts after NYSC."
- Do not invent information. If the tool returns no info, say so.

Rule of thumb: If the user is asking for information (not just chatting), retrieve first.

When you retrieve documents, cite them in your answer. If documents don't contain the answer, say so.
""")

print("✅ System prompt configured")

✅ System prompt configured


**Define Agent Nodes**

In [31]:
tools = [retrieve_nelfund_docs]
llm_with_tools = llm.bind_tools(tools)

def assistant(state: MessagesState) -> dict:
    """
    Assistant node - decides whether to retrieve or answer directly.
    """
    messages = [system_prompt] + state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """
    Decide whether to call tools or finish.
    """
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"
    return "__end__"
print("✅ Agent nodes defined")

✅ Agent nodes defined


**Build the Graph**

In [32]:
# Build graph
builder = StateGraph(MessagesState)

# Add nodes
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    should_continue,
    {"tools": "tools", "__end__": END}
)
builder.add_edge("tools", "assistant")

# Add memory
memory = MemorySaver()
agent = builder.compile(checkpointer=memory)

print("✅ Agentic RAG system compiled")

✅ Agentic RAG system compiled


In [33]:
def query_agent(user_input: str, thread_id: str = "default_session"):
    """
    Improved query function with clearer output.
    """
    print(f"\n{'='*70}")
    print(f"👤 User: {user_input}")
    print(f"{'='*70}\n")

    result = agent.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config={"configurable": {"thread_id": thread_id}}
    )

    # Check what happened
    used_retrieval = False
    final_answer = None

    for message in result["messages"]:
        if isinstance(message, AIMessage):
            if message.tool_calls:
                used_retrieval = True
                print(f"🔍 Agent: [Calling retrieval tool...]")
            if message.content and not message.tool_calls:
                final_answer = message.content

    # Always print final answer
    if final_answer:
        print(f"🤖 Agent: {final_answer}")
    else:
        print(f"⚠️ No response generated after retrieval!")

    # Summary
    print(f"\n📊 Decision: {'USED RETRIEVAL' if used_retrieval else 'ANSWERED DIRECTLY'}")
    print(f"{'='*70}\n")



In [35]:
query_agent("How does NELFUND works?", thread_id="session_0")


👤 User: How does NELFUND works?

🔍 Agent: [Calling retrieval tool...]
🤖 Agent: NELFUND (National Education and Learning Fund) works by providing financial support to students in Nigeria who face challenges in funding their higher education. Here are the key components of how NELFUND operates:

1. **Enhancing Access to Higher Education**: NELFUND aims to broaden access to higher education by offering loans to eligible students, ensuring that financial constraints do not hinder their educational pursuits.

2. **Financial Inclusivity**: The program promotes equal opportunities for all eligible students, aiming to eliminate discrimination based on financial background.

3. **Skill Development**: NELFUND supports the acquisition of useful skills, contributing to the development of a skilled workforce that meets the demands of the job market.

4. **Socio-economic Mobility**: By providing financial assistance, NELFUND seeks to create opportunities for students from various socio-economic bac

In [36]:
query_agent("How do I apply?", thread_id="session_0")


👤 User: How do I apply?

🔍 Agent: [Calling retrieval tool...]
🔍 Agent: [Calling retrieval tool...]
🤖 Agent: To apply for a NELFUND loan, follow these steps:

1. **Complete the Online Application**: 
   - Visit the NELFUND website at [nelf.gov.ng](http://nelf.gov.ng) and fill out the online application form.
   - Provide accurate personal information, academic records, and any other necessary documentation as required.

2. **Indemnity Agreement**: 
   - Complete an online indemnity agreement for the verification of the data submitted.

3. **Documentation Requirements**: 
   - Attach the necessary documents to your application, including:
     - A valid admission letter or Student ID.

4. **Submission Channels**: 
   - While the primary method is through the online portal, the NELFUND Board may determine alternate channels for submission depending on circumstances.

Make sure to check the website for any updates or additional requirements that may be needed during the application proces